In [20]:
import json
import requests
import pandas as pd
from datetime import datetime

def getLatestData():
    stock_dic = [
        { "stock_name" : "KDX", "stock_no" : "226490" },
        { "stock_name" : "KDI", "stock_no" : "114800" }
    ]

    df_result = None
    for stock in stock_dic:
        url = "http://www.paxnet.co.kr/stock/analysis/chart?abbrSymbol={}".format(stock["stock_no"])
        res = requests.get(url)
        result = res.text.split("var sise60minJson = \'")[1].split("\';\r\n \t")[0]
        json_res = json.loads(result)
        df = pd.DataFrame(json_res["body"]["chartDataList"])
        df["ymd"] = df["tradeDt"].apply(lambda x: str(datetime.fromtimestamp(x)))
        df["DATE"] = df["ymd"].str[:10]
        df["TIME"] = df["ymd"].str[11:13]
        df["DATETIME"] = df["DATE"].str.replace("-", "") + df["TIME"]
        df.drop(columns=["symbol", "tradeDt", "ymd"], inplace=True)
        df.rename(columns={
            "openPrice" : stock["stock_name"] + "_OPN",
            "highPrice" : stock["stock_name"] + "_HIGH",
            "lowPrice" : stock["stock_name"] + "_LOW",
            "closePrice" : stock["stock_name"] + "_CLS",
            "volume" : stock["stock_name"] + "_VOL",
        }, inplace=True)
        df["TIME"] = df["TIME"].astype("int64")
        df["DATETIME"] = df["DATETIME"].astype("int64")
        df[stock["stock_name"] + "_VOL"] = df[stock["stock_name"] + "_VOL"].astype("int64")
        df = df[["DATE", "TIME", "DATETIME", stock["stock_name"] + "_OPN", stock["stock_name"] + "_CLS", stock["stock_name"] + "_HIGH", stock["stock_name"] + "_LOW", stock["stock_name"] + "_VOL"]]
        if df_result is None:
            df_result = df
        else:
            df.drop(columns=["DATE", "TIME"], inplace=True)
            df_result = df_result.merge(df, how="inner", on="DATETIME")
    return df_result

In [21]:
df = getLatestData()
df

,DATE,TIME,DATETIME,KDX_OPN,KDX_CLS,KDX_HIGH,KDX_LOW,KDX_VOL,KDI_OPN,KDI_CLS,KDI_HIGH,KDI_LOW,KDI_VOL
0,2021-12-23,15,2021122315,29663,29693,29693,29629,4508,4020,4010,4025,4010,1826831
1,2021-12-23,16,2021122316,29693,29688,29717,29678,3059,4010,4005,4015,4005,1716988
2,2021-12-24,10,2021122410,29781,29934,30008,29781,12299,3995,3975,3995,3960,14110978
3,2021-12-24,11,2021122411,29934,29900,29959,29855,11749,3980,3980,3985,3970,2705624
4,2021-12-24,12,2021122412,29875,29865,29890,29836,24455,3985,3980,3990,3975,1658640
...,...,...,...,...,...,...,...,...,...,...,...,...,...
619,2022-05-06,12,2022050612,26685,26685,26695,26660,22277,4480,4480,4490,4475,1808380
620,2022-05-06,13,2022050613,26690,26720,26720,26685,2714,4480,4475,4485,4470,3089548
621,2022-05-06,14,2022050614,26720,26725,26740,26715,4671,4475,4470,4480,4465,1454606
622,2022-05-06,15,2022050615,26725,26730,26730,26710,3969,4475,4470,4480,4470,2317011


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 624 entries, 0 to 623
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   DATE      624 non-null    object
 1   TIME      624 non-null    int64 
 2   DATETIME  624 non-null    int64 
 3   KDX_OPN   624 non-null    int64 
 4   KDX_CLS   624 non-null    int64 
 5   KDX_HIGH  624 non-null    int64 
 6   KDX_LOW   624 non-null    int64 
 7   KDX_VOL   624 non-null    int64 
 8   KDI_OPN   624 non-null    int64 
 9   KDI_CLS   624 non-null    int64 
 10  KDI_HIGH  624 non-null    int64 
 11  KDI_LOW   624 non-null    int64 
 12  KDI_VOL   624 non-null    int64 
dtypes: int64(12), object(1)
memory usage: 68.2+ KB
